In [ ]:
# This is the application of the DNN network from (Kell et al., 2018), we want to pass the stimuli from MiND auditoty fMRI tasks to the DNN
# and further analyze the activation characteristics in the DNN
# written by Violet Sept.2024

In [1]:
import IPython.display as ipd
import sys
sys.path.append('C:\\Users\\quanz\\Documents\\UM\\Projects\\GLX_Project\\DNN\\kelletal2018-master\\kelletal2018-master\\network')
from branched_network_class import branched_network
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import scipy.io.wavfile as wav
import matplotlib as plt 
%pylab inline

Instructions for updating:
non-resource variables are not supported in the long term
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


C:\Users\quanz\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
# import the following to run demo_from_wav()
import os
new_directory = ('C:/Users/quanz/Documents/UM/Projects/GLX_Project/DNN/kelletal2018-master/kelletal2018-master')
os.chdir(new_directory)
#sys.path.append(new_directory)
from pycochleagram import cochleagram as cgram 
from PIL import Image

C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\kelletal2018-master\kelletal2018-master\pycochleagram\erbfilter.py:8: RuntimeWarning: pycochleagram using non-interactive Agg matplotlib backend
  from pycochleagram import utils


In [3]:
## Some helper functions
def resample(example, new_size):
    im = Image.fromarray(example)
    resized_image = im.resize(new_size, resample=Image.ANTIALIAS)
    return np.array(resized_image)

def plot_cochleagram(cochleagram, title): 
    plt.figure(figsize=(6,3))
    plt.matshow(cochleagram.reshape(256,256), origin='lower',cmap=plt.cm.Blues, fignum=False, aspect='auto')
    plt.yticks([]); plt.xticks([]); plt.title(title); 
    
def play_wav(wav_f, sr, title):   
    print(title+':')
    ipd.display(ipd.Audio(wav_f, rate=sr))

In [4]:
def demo_pre_generated_cochleagram():
    tf.reset_default_graph()

    net_object = branched_network() # make network object
    word_key = np.load('./demo_stim/logits_to_word_key.npy', encoding='bytes', allow_pickle=True) #Load logits to word key 
    music_key = np.load('./demo_stim/logits_to_genre_key.npy',encoding='bytes', allow_pickle=True) #Load logits to genre key

    # example pre-generated speech cochleagram  
    example_cochleagram = np.load('./demo_stim/example_cochleagram_0.npy') 
    plot_cochleagram(example_cochleagram,'Example speech cochleagram' )

    # run cochleagram through network and get logits for word branch
    logits = net_object.session.run(net_object.word_logits, feed_dict={net_object.x: example_cochleagram})

    # determine word branch prediction 
    prediction = word_key[np.argmax(logits, axis=1)]
    print("Speech Example ... actual label: according  predicted_label: " + prediction[0].decode('utf-8') +'\n')
    
    # example pre-generated music cochleagram
    example_cochleagram_music = np.load('./demo_stim/example_cochleagram_1.npy') 
    plot_cochleagram(example_cochleagram_music,'Example music cochleagram' )
    
    # run cochleagram through network and get logits for genre branch
    logits_music = net_object.session.run(net_object.genre_logits, 
                                          feed_dict={net_object.x: example_cochleagram_music})
    # note: throughout paper top-5 accuracy is reported for genre task
    prediction_music = (logits_music).argsort()[:,-5:][0][::-1] 
    print("Music Example... actual label: "+ music_key[11].decode('utf-8') + "  top-5 predicted_labels (in order of confidence): ")
    print("\n"+ "; ".join(key.decode('utf-8') for key in music_key[prediction_music]))


In [5]:
def generate_cochleagram(wav_f, sr, title):
    # Define parameters
    n, sampling_rate = 50, 16000
    low_lim, hi_lim = 20, 8000
    sample_factor, pad_factor, downsample = 4, 2, 200
    nonlinearity, fft_mode, ret_mode = 'power', 'auto', 'envs'
    strict = True

    # Create cochleagram
    c_gram = cgram.cochleagram(wav_f, sr, n, low_lim, hi_lim, 
                               sample_factor, pad_factor, downsample,
                               nonlinearity, fft_mode, ret_mode, strict)
    
    # Debugging statements
    print("Cochleagram shape:", c_gram.shape)
    print("Min value in cochleagram:", np.min(c_gram))
    print("Max value in cochleagram:", np.max(c_gram))

    # Avoiding log10 issues
    # c_gram[c_gram <= 0] = 1e-10  # Set non-positive values to a small positive number

    # Rescale to [0, 255]
    # c_gram_rescaled = 255 * (1 - ((np.max(c_gram) - c_gram) / np.ptp(c_gram)))

    # Check the shape of c_gram_rescaled
    # print("Shape of c_gram_rescaled:", c_gram_rescaled.shape)

     # rescale to [0,255]
    c_gram_rescaled =  255*(1-((np.max(c_gram)-c_gram)/np.ptp(c_gram)))
    
    # reshape to (256,256)
    c_gram_reshape_1 = np.reshape(c_gram_rescaled, (211,400))
    c_gram_reshape_2 = resample(c_gram_reshape_1,(256,256))
    
    plot_cochleagram(c_gram_reshape_2, title)

    # prepare to run through network -- i.e., flatten it
    c_gram_flatten = np.reshape(c_gram_reshape_2, (1, 256*256)) 
    
    return c_gram_flatten


In [6]:
import numpy as np
from scipy.io import wavfile as wav
import librosa

def demo_from_MiND():
    # tf.reset_default_graph()  # Reset the TensorFlow graph

    global net_object
    global c_gram
    net_object = branched_network()  # Initialize your network model

    # Load the keys for the model outputs
    word_key = np.load('./demo_stim/logits_to_word_key.npy')  # Speech labels
    music_key = np.load('./demo_stim/logits_to_genre_key.npy')  # Music genre labels

    # Replace the speech example with your Violin.wav stimulus
    # file_path = r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\Test\Violin.wav'
    # sr, wav_f = wav.read('./demo_stim/example_6.wav')
    
    # Generate the cochleagram for Violin.wav and pass it through the network
    # sr, wav_f = wav.read('./demo_stim/example_6.wav')
    sr, wav_f = wav.read(r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\Test\Word\Segment_3.wav')

    # Resample the audio to 16,000 Hz using librosa
    # wav_f = librosa.resample(wav_f_org.astype(float), orig_sr=original_sr, target_sr=16000)
    
    # Check the shape and convert to mono if stereo
    if wav_f.ndim == 2:  # Check if it's stereo
        wav_f = np.mean(wav_f, axis=1).astype(np.int16)  # Average the two channels to create a mono signal

    play_wav(wav_f, sr, 'Example')  # Play Violin.wav (if required)

    # Generate cochleagram
    c_gram = generate_cochleagram(wav_f, sr, 'Example')  # Assuming generate_cochleagram is defined

    # Get logits for word branch (if classifying music)
    # logits = net_object.session.run(net_object.genre_logits, feed_dict={net_object.x: c_gram})
    # prediction = (logits).argsort()[:,-5:][0][::-1] 
    # print("Music Example ... \n Background: Auditory Scene, snr: -3db, actual label: " \
    #     + music_key[1].decode('utf-8') + ",\n top-5 predicted_labels (in order of confidence): \n " \
    #     + ";\n ".join(key.decode('utf-8') for key in music_key[prediction]) + "\n")

    # Get logits for word branch (if classifying speech)
    logits = net_object.session.run(net_object.word_logits, feed_dict={net_object.x: c_gram})
    prediction = word_key[np.argmax(logits, axis=1)]
    print("Speech Example ... \n background: speaker shaped noise, snr: -3db, actual label: Washington, predicted_label: " \
        + prediction[0].decode('utf-8') +'\n')
    

demo_from_MiND()

In [ ]:
file_path = r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\Test\Violin.wav'
sr, wav_f = wav.read(file_path)  # sr = sampling rate, wav_f = audio signal

print("Shape of wav_f:", wav_f.shape)
print("Data type of wav_f:", wav_f.dtype)
print(f"Sampling Rate: {sr} Hz")

In [ ]:
pip install librosa

In [27]:

from branched_network_class import branched_network

# Create an instance of the network
net_object = branched_network()

# Prepare your input data (ensure it's the correct shape)
input_data = c_gram  # Shape should be (batch_size, 256 * 256), for example

# Run the session to get activations
activations =  net_object.get_activations(input_data)

# Print the shapes of activations from each layer
for layer_name, activation in activations.items():
    print(f"{layer_name} activation shape: {activation.shape}")

conv1 activation shape: (1, 86, 86, 96)
rnorm1 activation shape: (1, 86, 86, 96)
pool1 activation shape: (1, 43, 43, 96)
conv2 activation shape: (1, 22, 22, 256)
rnorm2 activation shape: (1, 22, 22, 256)
pool2 activation shape: (1, 11, 11, 256)
conv3 activation shape: (1, 11, 11, 512)
conv4_W activation shape: (1, 11, 11, 1024)
conv5_W activation shape: (1, 11, 11, 512)
pool5_W activation shape: (1, 6, 6, 512)
conv4_G activation shape: (1, 11, 11, 1024)
conv5_G activation shape: (1, 11, 11, 512)
pool5_G activation shape: (1, 6, 6, 512)


In [43]:
import matplotlib.pyplot as plt

# Example to visualize the first channel of the first layer's activation
layer_to_visualize = 'pool5_G'  # Change to the layer you want to visualize
activation = activations[layer_to_visualize]

# Select the first filter's output (assuming a single batch size)
plt.imshow(activation[0, :, :, 0], cmap='viridis')
plt.title(f"Activation from {layer_to_visualize}")
plt.colorbar()

# Save the figure to a file
plt.savefig(f"{layer_to_visualize}_activation.png")  # Save as a PNG file
plt.close()  # Close the figure after saving

In [32]:
print(os.getcwd())


C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\kelletal2018-master\kelletal2018-master
